In [3]:
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [7]:
import os
from dotenv import load_dotenv

# 1. Load the variables from the .env file
load_dotenv()

# 2. The API key is now loaded into the environment variables (os.environ)
#    Now you can access it, and configure the SDK.

# Using the new, recommended SDK:
from google.genai import Client

# The Client will automatically look for GOOGLE_API_KEY in os.environ
# because you called load_dotenv() earlier.
client = Client()

# If you prefer to explicitly pass it (less common with the new SDK):
# client = Client(api_key=os.environ.get("GOOGLE_API_KEY"))

# If you were still using the old SDK (but you should switch!):
# import google.generativeai as genai
# genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))

print("API Key configured successfully using the .env file.")

API Key configured successfully using the .env file.


In [9]:
def show_python_code_and_result(response):
    """
    Parses a sequence of Gemini API response objects (often 'response.candidates')
    to extract and print generated Python code and its execution result, 
    specifically from a tool call (like the Code Executor).
    """
    for i in range(len(response)):
        # Iterate through all parts of the multi-part response
        
        # Check if the response contains a valid function call result from the code executor
        if (
            # 1. Ensure the current part of the response has content
            (response[i].content.parts)
            # 2. Ensure the first part of the content exists
            and (response[i].content.parts[0])
            # 3. Ensure the first part contains a function_response object (meaning a tool was called)
            and (response[i].content.parts[0].function_response)
            # 4. Ensure the function_response has the actual response data
            and (response[i].content.parts[0].function_response.response)
        ):
            # Extract the response data dictionary from the Code Executor tool
            response_code = response[i].content.parts[0].function_response.response
            
            # Check if the 'result' key exists and its value is not just a placeholder "```"
            if "result" in response_code and response_code["result"] != "```":
                # Check for a specific marker ("tool_code") indicating the content is the code itself
                if "tool_code" in response_code["result"]:
                    print(
                        # Print the generated Python code, removing the "tool_code" marker for clean output
                        "Generated Python Code >> ",
                        response_code["result"].replace("tool_code", ""),
                    )
                else:
                    # If "tool_code" isn't present, the content is likely the output/result of the code execution
                    print("Generated Python Response >> ", response_code["result"])


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [11]:
# 1.5: Configure Retry Options
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [13]:
# 2.1: Building Custom Function Tools
# This section demonstrates how to define a Python function that the Gemini model
# can call as a tool (often referred to as a function calling or tool use capability).

# Example: Currency Converter Agent
# This specific example creates a tool for an agent that might need to calculate fees.

def get_fee_for_payment_method(method: str) -> dict:
    """Looks up the transaction fee percentage for a given payment method.

    This tool simulates looking up a company's internal fee structure based on
    the name of the payment method provided by the user.

    Args:
        method: The name of the payment method. It should be descriptive,
                e.g., "platinum credit card" or "bank transfer".
                This parameter is used by the model when it decides to call the tool.

    Returns:
        Dictionary with status and fee information.
        Success: {"status": "success", "fee_percentage": 0.02}
        Error: {"status": "error", "error_message": "Payment method not found"}
        The model uses this returned dictionary to formulate its final answer.
    """
     if not isinstance(method, str):
         raise TypeError("method must be a string")
    # This simulates looking up a company's internal fee structure.
    fee_database = {
        # Define a dictionary acting as a simple database of payment methods and their fees.
        "platinum credit card": 0.02,  # 2%
        "gold debit card": 0.035,  # 3.5%
        "bank transfer": 0.01,  # 1%
    }

    # Retrieve the fee, converting the input method to lowercase for case-insensitive lookup.
    fee = fee_database.get(method.lower())
    
    # Check if the payment method was found in the database.
    if fee is not None:
        # Success case: return a dictionary indicating success and the fee percentage.
        return {"status": "success", "fee_percentage": fee}
    else:
        # Error case: return a dictionary indicating an error and a user-friendly message.
        return {
            "status": "error",
            "error_message": f"Payment method '{method}' not found",
        }


# Confirmation that the tool function has been successfully defined.
print("✅ Fee lookup function created")
# Test the function directly to verify its output format and logic.
print(f"💳 Test: {get_fee_for_payment_method('platinum credit card')}")

✅ Fee lookup function created
💳 Test: {'status': 'success', 'fee_percentage': 0.02}


In [15]:
def get_exchange_rate(base_currency: str, target_currency: str) -> dict:
    """Looks up and returns the exchange rate between two currencies.

    This function serves as a custom tool that a Gemini agent can call
    to retrieve real-time or static financial data needed to answer a user's request.

    Args:
        base_currency: The ISO 4217 currency code of the currency you
                       are converting from (e.g., "USD").
                       The model must correctly identify this argument from the user's prompt.
        target_currency: The ISO 4217 currency code of the currency you
                         are converting to (e.g., "EUR").
                         The model must correctly identify this argument from the user's prompt.

    Returns:
        Dictionary with status and rate information. This is the output
        the model uses to continue its reasoning or generate a final response.
        Success: {"status": "success", "rate": 0.93}
        Error: {"status": "error", "error_message": "Unsupported currency pair"}
    """

    # Static data simulating a live exchange rate API
    # IMPORTANT: In a production environment, this should be replaced with a 
    # call to a reliable external API (e.g., requests.get("api.exchangerates.com")).
    rate_database = {
        # The keys here are the lowercased base currencies.
        "usd": {
            # Nested dictionary contains target currencies and their rates (e.g., 1 USD = X EUR).
            "eur": 0.93,  # Euro
            "jpy": 157.50,  # Japanese Yen
            "inr": 83.58,  # Indian Rupee
        }
        # Add more base currencies (e.g., "eur": {...}) as needed.
    }

    # Input validation and processing
    # Convert both currency codes to lowercase for case-insensitive lookup in the database.
    base = base_currency.lower()
    target = target_currency.lower()

    # Return structured result with status
    # This line uses dictionary .get() methods to safely navigate the nested structure:
    # 1. Get the inner dictionary for the base currency (e.g., rate_database["usd"]).
    # 2. On that inner dictionary, get the rate for the target currency (e.g., inner_dict["eur"]).
    # If any key is missing, .get() returns None.
    rate = rate_database.get(base, {}).get(target)
    
    # Check if a valid rate was found.
    if rate is not None:
        # Success case: Return the rate with a 'success' status.
        return {"status": "success", "rate": rate}
    else:
        # Error case: Return an error message if the currency pair is not in the static data.
        return {
            "status": "error",
            "error_message": f"Unsupported currency pair: {base_currency}/{target_currency}",
        }


# Outputting confirmation and a test run to show the function is defined and works.
print("✅ Exchange rate function created")
print(f"💱 Test: {get_exchange_rate('USD', 'EUR')}")

✅ Exchange rate function created
💱 Test: {'status': 'success', 'rate': 0.93}


In [17]:
# Create an LlmAgent instance, which represents an autonomous agent powered by the Gemini model.
currency_agent = LlmAgent(
    # Assign a unique, descriptive name to the agent.
    name="currency_agent",
    # Configure the underlying language model (LLM) for the agent.
    model=Gemini(
        model="gemini-2.5-flash-lite", # Specify the model version to use (e.g., a fast, light model).
        retry_options=retry_config     # Apply configuration for retrying failed API calls.
    ),
    # Provide the agent with its system instruction (persona, goals, and required steps).
    instruction="""You are a smart currency conversion assistant.

    For currency conversion requests:
    1. Use `get_fee_for_payment_method()` to find transaction fees
    2. Use `get_exchange_rate()` to get currency conversion rates
    3. Check the "status" field in each tool's response for errors
    4. Calculate the final amount after fees based on the output from `get_fee_for_payment_method` and `get_exchange_rate` methods and provide a clear breakdown.
    5. First, state the final converted amount.
        Then, explain how you got that result by showing the intermediate amounts. Your explanation must include: the fee percentage and its
        value in the original currency, the amount remaining after the fee, and the exchange rate used for the final conversion.

    If any tool returns status "error", explain the issue to the user clearly.
    """,
    # Register the custom Python functions as tools the agent can call.
    # The model's reasoning engine will use the docstrings and signatures of these functions
    # to decide when and how to call them.
    tools=[get_fee_for_payment_method, get_exchange_rate],
)

# Output confirmation and list the tools available to the newly created agent.
print("✅ Currency agent created with custom function tools")
print("🔧 Available tools:")
print("  • get_fee_for_payment_method - Looks up company fee structure")
print("  • get_exchange_rate - Gets current exchange rates")

✅ Currency agent created with custom function tools
🔧 Available tools:
  • get_fee_for_payment_method - Looks up company fee structure
  • get_exchange_rate - Gets current exchange rates


In [19]:
# Initialize the InMemoryRunner, which is the mechanism used to execute the LlmAgent.
# The runner manages the conversational flow, including the agent's calls to its registered tools.
currency_runner = InMemoryRunner(
    agent=currency_agent # Pass the previously defined 'currency_agent' to the runner.
)

# Execute the agent with a specific user prompt using the run_debug method.
# The 'await' keyword is used because 'run_debug' is an asynchronous function.
# The result is assigned to '_' because we are primarily interested in the side effects 
# (the print statements from the agent's thought process and the final output) 
# printed to the console in debug mode.
_ = await currency_runner.run_debug(
    # The user's query that triggers the agent's logic, including tool selection and calculation.
    "I want to convert 500 US Dollars to Euros using my Platinum Credit Card. How much will I receive?"
)

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "C:\Users\SERGE\Documents\WGU\5-Days_Agent_development\DAY_2_Agent Tools_Interoperability with Model Context Protocol\kaggle-ai-agentDay2a-AgentTools\.venv\Lib\site-packages\google\adk\agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > I want to convert 500 US Dollars to Euros using my Platinum Credit Card. How much will I receive?


currency_agent > I will convert 500 USD to EUR for you.

First, a 2.0% fee will be deducted, amounting to 10.0 USD. This leaves you with 490.0 USD.
The exchange rate from USD to EUR is 0.93.
Therefore, you will receive 455.7 EUR.


#  Section 3: Improving Agent Reliability with Code

In [24]:
#3.1 Built-in Code Executor

calculation_agent = LlmAgent(
    name="CalculationAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a specialized calculator that ONLY responds with Python code. You are forbidden from providing any text, explanations, or conversational responses.
 
     Your task is to take a request for a calculation and translate it into a single block of Python code that calculates the answer.
     
     **RULES:**
    1.  Your output MUST be ONLY a Python code block.
    2.  Do NOT write any text before or after the code block.
    3.  The Python code MUST calculate the result.
    4.  The Python code MUST print the final result to stdout.
    5.  You are PROHIBITED from performing the calculation yourself. Your only job is to generate the code that will perform the calculation.
   
    Failure to follow these rules will result in an error.
       """,
    code_executor=BuiltInCodeExecutor(),  # Use the built-in Code Executor Tool. This gives the agent code execution capabilities
)

In [26]:
enhanced_currency_agent = LlmAgent(
    name="enhanced_currency_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    # Updated instruction
    instruction="""You are a smart currency conversion assistant. You must strictly follow these steps and use the available tools.

  For any currency conversion request:

   1. Get Transaction Fee: Use the get_fee_for_payment_method() tool to determine the transaction fee.
   2. Get Exchange Rate: Use the get_exchange_rate() tool to get the currency conversion rate.
   3. Error Check: After each tool call, you must check the "status" field in the response. If the status is "error", you must stop and clearly explain the issue to the user.
   4. Calculate Final Amount (CRITICAL): You are strictly prohibited from performing any arithmetic calculations yourself. You must use the calculation_agent tool to generate Python code that calculates the final converted amount. This 
      code will use the fee information from step 1 and the exchange rate from step 2.
   5. Provide Detailed Breakdown: In your summary, you must:
       * State the final converted amount.
       * Explain how the result was calculated, including:
           * The fee percentage and the fee amount in the original currency.
           * The amount remaining after deducting the fee.
           * The exchange rate applied.
    """,
    tools=[
        get_fee_for_payment_method,
        get_exchange_rate,
        AgentTool(agent=calculation_agent),  # Using another agent as a tool!
    ],
)

print("✅ Enhanced currency agent created")
print("🎯 New capability: Delegates calculations to specialist agent")
print("🔧 Tool types used:")
print("  • Function Tools (fees, rates)")
print("  • Agent Tool (calculation specialist)")

✅ Enhanced currency agent created
🎯 New capability: Delegates calculations to specialist agent
🔧 Tool types used:
  • Function Tools (fees, rates)
  • Agent Tool (calculation specialist)


In [30]:
#defining a runner
enhanced_runner = InMemoryRunner(agent=enhanced_currency_agent)

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "C:\Users\SERGE\Documents\WGU\5-Days_Agent_development\DAY_2_Agent Tools_Interoperability with Model Context Protocol\kaggle-ai-agentDay2a-AgentTools\.venv\Lib\site-packages\google\adk\agents", which implies app name "agents".


In [32]:
response = await enhanced_runner.run_debug(
    "Convert 1,250 USD to INR using a Bank Transfer. Show me the precise calculation."
)


 ### Created new session: debug_session_id

User > Convert 1,250 USD to INR using a Bank Transfer. Show me the precise calculation.


App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "C:\Users\SERGE\Documents\WGU\5-Days_Agent_development\DAY_2_Agent Tools_Interoperability with Model Context Protocol\kaggle-ai-agentDay2a-AgentTools\.venv\Lib\site-packages\google\adk\agents", which implies app name "agents".


enhanced_currency_agent > The conversion of 1,250 USD to INR using a Bank Transfer results in 103,659.75 INR.

Here's the breakdown of the calculation:
*   **Bank Transfer Fee:** A 1% fee on 1,250 USD amounts to 12.50 USD.
*   **Amount after Fee Deduction:** After deducting the fee, 1,237.50 USD remains.
*   **Exchange Rate:** The exchange rate applied is 83.58 INR per USD.
*   **Final Converted Amount:** The remaining 1,237.50 USD is converted to 103,659.75 INR.


In [34]:
show_python_code_and_result(response)

Generated Python Code >>  
original_amount_usd = 1250
exchange_rate_inr_per_usd = 83.58
fee_percentage = 0.01

fee_amount_usd = original_amount_usd * fee_percentage
amount_after_fee_usd = original_amount_usd - fee_amount_usd
final_amount_inr = amount_after_fee_usd * exchange_rate_inr_per_usd

print(f"Original Amount (USD): {original_amount_usd}")
print(f"Bank Transfer Fee (USD): {fee_amount_usd:.2f}")
print(f"Amount after Fee Deduction (USD): {amount_after_fee_usd:.2f}")
print(f"Exchange Rate (INR/USD): {exchange_rate_inr_per_usd}")
print(f"Final Converted Amount (INR): {final_amount_inr:.2f}")

